<h1>개인과제</h1>
<h2>주피터랩에서 데이터를 입력받으면 3가지 DB에 동시에 CRUD가 되는 시스템을 만들어라</h2>
<h3>예) input을 이용하여 <이름:> 홍길동(엔터), <나이:> 33(엔터),<주소:> 해운대구(엔터)</h3>
<h3>입력하고, 데이터를 검색하고 이름을 고르고 수정하고 삭제 가능하게 하라</h3>

In [ ]:
def insert_oracle(name, age, address):
    import oracledb
    try:
        conn = oracledb.connect(
            user='pknu',
            password='1234',
            dsn='oracle/XEPDB1'
        )
        print('[Oracle] 연결 성공')
        cursor = conn.cursor()
        
        query = """
            BEGIN
                EXECUTE IMMEDIATE'
                    CREATE TABLE students2(
                        id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
                        name VARCHAR2(50),
                        age VARCHAR2(50),
                        address VARCHAR2(100)
                    )';
            EXCEPTION
                WHEN OTHERS THEN
                    IF SQLCODE != -955 THEN
                        RAISE;
                    END IF;
            END;
        """
        cursor.execute(query)
        print('[Oracle] 테이블 확인/생성 완료')

        que = 'INSERT INTO students2(name, age, address) VALUES (:1, :2, :3)'
        cursor.execute(que, (name, age, address))
        conn.commit()
        print('[Oracle] 데이터 삽입 완료')

    except Exception as e:
        print('[Oracle] 오류:', e)
    finally:
        if conn:
            conn.close()

def insert_mongodb(name, age, city):
    from pymongo import MongoClient
    try:
        client = MongoClient('mongodb://mongodb:27017')
        db = client['pknu']
        collection = db['students2']
        print('[MongoDB] 연결 및 컬렉션 선택 완료')
        user = {'name': name, 'age': age, 'city': city}
        result = collection.insert_one(user)
        print('[MongoDB] 삽입 완료, ID:', result.inserted_id)
    except Exception as e:
        print('[MongoDB] 오류:', e)

def insert_mysql(name, age, city):
    import mysql.connector
    try:
        conn = mysql.connector.connect(
            host="mysql",
            user="root",
            password="1234"
        )
        cursor = conn.cursor()
        cursor.execute('CREATE DATABASE IF NOT EXISTS pknu')
        conn.close()

        conn = mysql.connector.connect(
            host="mysql",
            user="root",
            password="1234",
            database="pknu"
        )
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS students2(
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(50),
                age INT,
                city VARCHAR(50)
            )
        ''')
        print('[MySQL] 테이블 확인/생성 완료')

        sql = 'INSERT INTO students2(name, age, city) VALUES (%s, %s, %s)'
        cursor.execute(sql, (name, age, city))
        conn.commit()
        print('[MySQL] 삽입 완료, ID:', cursor.lastrowid)
    except Exception as e:
        print('[MySQL] 오류:', e)
    finally:
        if conn:
            conn.close()

def select_oracle():
    import oracledb
    try:
        conn = oracledb.connect(
            user='pknu',
            password='1234',
            dsn='oracle/XEPDB1'
        )
        cursor = conn.cursor()
        que = 'SELECT * FROM students2'
        cursor.execute(que)
        students = cursor.fetchall()
        print('학생목록: ')
        for s in students:
            print(s)
    except Exception as e:
        print('[Oracle] 읽기 오류:', e)
    finally:
        if conn:
            conn.close()

def select_mongodb():
    from pymongo import MongoClient
    try:
        client = MongoClient('mongodb://mongodb:27017')
        db = client['pknu']
        collection = db['students2']
        # docs = collection.find()
        users = collection.find({},{'_id':0})
        print('\n[MongoDB] 학생 목록:')
        for user in users:
            print(user)
    except Exception as e:
        print('[MongoDB] 읽기 오류:', e)

def select_mysql():
    import mysql.connector
    try:
        conn = mysql.connector.connect(
            host="mysql",
            user="root",
            password="1234",
            database="pknu"
        )
        cursor = conn.cursor()
        que = 'SELECT * FROM students2'
        cursor.execute(que)
        users = cursor.fetchall()
        print('\n[MySQL] 학생 목록:')
        for user in users:
            print(user)
    except Exception as e:
        print('[MySQL] 읽기 오류:', e)
    finally:
        if conn:
            conn.close()

def delete_oracle(inp5):
    import oracledb
    try:
        conn = oracledb.connect(
            user='pknu',
            password='1234',
            dsn='oracle/XEPDB1'
        )
        cursor = conn.cursor()
        que = 'DELETE FROM students2 WHERE name = :1'
        values = (inp5,)
        cursor.execute(que,values)
        conn.commit()
        print('데이터가 삭제되었습니다')
    except Exception as e:
        print('[Oracle] 삭제 오류:', e)
    finally:
        if conn:
            conn.close()

def delete_mongodb(inp5):
    from pymongo import MongoClient
    try:
        client = MongoClient('mongodb://mongodb:27017')
        db = client['pknu']
        collection = db['students2']
        delete = collection.delete_one({'name':inp5})
        print(delete)
    except Exception as e:
        print('[MongoDB] 삭제오류:', e)

def delete_mysql(inp5):
    import mysql.connector
    try:
        conn = mysql.connector.connect(
            host="mysql",
            user="root",
            password="1234",
            database="pknu"
        )
        cursor = conn.cursor()
        que = 'DELETE FROM students2 WHERE name = %s'
        filters = [inp5]
        cursor.execute(que, filters)
        conn.commit()
    except Exception as e:
        print('[MySQL] 삭제 오류:', e)
    finally:
        if conn:
            conn.close()




str = """
1. 데이터 등록
2. 데이터 읽기
3. 데이터 수정
4. 데이터 삭제
5. 종료
고르세요
"""

while True:
    print(str)
    inp = input('선택하세요: ')
    if inp == '1':
        inp1 = input('이름을 입력하세요: ')
        inp2 = input('나이를 입력하세요: ')
        inp3 = input('주소를 입력하세요: ')

        insert_oracle(inp1, inp2, inp3)
        insert_mongodb(inp1, inp2, inp3)
        insert_mysql(inp1, inp2, inp3)
    if inp == '2' : 
        select_oracle()
        select_mongodb()
        select_mysql()
    if inp == '3' : pass
    if inp == '4' :
        inp5 = input('삭제할 이름을 입력하세요: ')

        delete_oracle(inp5)
        delete_mongodb(inp5)
        delete_mysql(inp5)
        
    elif inp == '5':
        print('종료합니다.')
        break
    else:
        print('잘못된 선택입니다.')



1. 데이터 등록
2. 데이터 읽기
3. 데이터 수정
4. 데이터 삭제
5. 종료
고르세요



선택하세요:  4
삭제할 이름을 입력하세요:  김파나


데이터가 삭제되었습니다
DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

1. 데이터 등록
2. 데이터 읽기
3. 데이터 수정
4. 데이터 삭제
5. 종료
고르세요

